### Conversión de archivo PHT a ONNX
Para el modelo resnet18

#### Función *save_resnet_model*
Esta función carga un modelo ResNet pre-entrenado con la función models.resnet18(pretrained=True) y luego guarda el diccionario de estado del modelo en un archivo .pth utilizando torch.save.

#### Función *pth_to_onnx*
Esta función convierte un modelo guardado en formato .pth a formato .onnx. Primero verifica si el archivo .onnx ya existe. Si no existe, carga el diccionario de estado del modelo .pth, crea una instancia del modelo ResNet sin pesos preentrenados, carga el diccionario de estado en esta instancia y finalmente exporta el modelo a formato .onnx.

In [4]:
import torch
import torchvision.models as models
import os
import onnx
# import onnxruntime

def save_resnet_model(model_name="resnet18"):
    # Cargar un modelo ResNet pre-entrenado
    model = models.resnet18(pretrained=True)
    
    # Guardar el modelo en un archivo .pth
    model_path = f"../models/{model_name}.pth"
    torch.save(model.state_dict(), model_path)
    
    print(f"Modelo ResNet pre-entrenado guardado en {model_path}")



def pth_to_onnx(model_name="resnet18"):
    # Verificar si el archivo ONNX ya existe
    onnx_path = f"../models/{model_name}.onnx"
    if os.path.isfile(onnx_path):
        print(f"El archivo {onnx_path} ya existe.")
        return

    # Ruta al modelo en la carpeta "models"
    model_path = f"../models/{model_name}.pth"

    # Verificar si el modelo especificado existe en la carpeta "models"
    if not os.path.isfile(model_path):
        print(f"No se encontró el archivo {model_name}.pth en la carpeta 'models'")
        print("Creando modelo")
        save_resnet_model(model_name)

    try:
        # Cargar el diccionario de estado del modelo
        state_dict = torch.load(model_path, map_location=torch.device('cpu'))

        # Crear una instancia del modelo
        model = models.resnet18(pretrained=False)  # No cargamos pesos preentrenados
        model.load_state_dict(state_dict)

        # Establecer el modelo en modo de evaluación
        model.eval()

        # Definir una entrada de ejemplo (tensor)
        dummy_input = torch.randn(1, 3, 224, 224)

        # Exportar el modelo a ONNX
        torch.onnx.export(model, dummy_input, onnx_path, export_params=True, opset_version=11)

        print(f"Modelo '{onnx_path}' exportado correctamente")
        
    except Exception as e:
        print(f"Error al convertir el modelo: {e}")
        raise


pth_to_onnx()

El archivo ../models/resnet18.onnx ya existe.


### Extracción información de archivos ONNX

#### Función *get_layers*
La función get_layers del módulo onnx_utils realiza las siguientes acciones:

1. Carga el modelo ONNX especificado.
2. Recorre el grafo del modelo para identificar y contar las capas.
3. Retorna un diccionario con los nombres de las capas como claves y su frecuencia como valores.

Puedes ejecutar este código para obtener rápidamente un resumen de las capas de un modelo en formato ONNX. Esto puede ser útil para entender la estructura del modelo antes de realizar otras operaciones como la inferencia o la conversión a otro formato.

In [5]:
def get_layers(onnx_model_path):
    # Cargar el modelo ONNX
    model = onnx.load(onnx_model_path)

    # Obtener el grafo del modelo
    graph = model.graph

    # Inicializar un diccionario para almacenar las capas y su frecuencia
    layers = {}

    # Iterar sobre las operaciones del grafo
    for node in graph.node:
        # Obtener el nombre de la capa y agregarlo al diccionario
        layer_name = node.op_type
        if layer_name in layers:
            layers[layer_name] += 1
        else:
            layers[layer_name] = 1

    return layers


model_layers = get_layers("../models/resnet18.onnx")
print("Diccionario de capas:")
print(model_layers)

Diccionario de capas:
{'Conv': 20, 'Relu': 17, 'MaxPool': 1, 'Add': 8, 'GlobalAveragePool': 1, 'Flatten': 1, 'Gemm': 1}


#### Función *get_layer_names*
La función get_layer_names del módulo onnx_utils realiza las siguientes acciones:

1. Carga el modelo ONNX especificado.
2. Recorre el grafo del modelo para extraer los nombres de todas las capas.
3. Retorna una lista con los nombres de todas las capas del modelo.

Puedes usar este código para obtener una lista completa de los nombres de las capas de un modelo en formato ONNX. Esta información es útil para tener una visión general de la estructura del modelo y puede ser especialmente útil antes de realizar operaciones más específicas o de depuración.

In [6]:
def get_layer_names(onnx_model_path):
    # Cargar el modelo ONNX
    model = onnx.load(onnx_model_path)

    # Obtener el grafo del modelo
    graph = model.graph

    # Inicializar una lista para almacenar los nombres de las capas
    layer_names = []

    # Iterar sobre las operaciones del grafo
    for node in graph.node:
        # Agregar el nombre de la operación a la lista de nombres de las capas
        layer_names.append(node.name)

    return layer_names

# Ejemplo de uso: obtener una lista de los nombres de todas las capas en el modelo ONNX
layer_names = get_layer_names("../models/resnet18.onnx")
print("Nombres de las capas:")
print(layer_names)

Nombres de las capas:
['/conv1/Conv', '/relu/Relu', '/maxpool/MaxPool', '/layer1/layer1.0/conv1/Conv', '/layer1/layer1.0/relu/Relu', '/layer1/layer1.0/conv2/Conv', '/layer1/layer1.0/Add', '/layer1/layer1.0/relu_1/Relu', '/layer1/layer1.1/conv1/Conv', '/layer1/layer1.1/relu/Relu', '/layer1/layer1.1/conv2/Conv', '/layer1/layer1.1/Add', '/layer1/layer1.1/relu_1/Relu', '/layer2/layer2.0/conv1/Conv', '/layer2/layer2.0/relu/Relu', '/layer2/layer2.0/conv2/Conv', '/layer2/layer2.0/downsample/downsample.0/Conv', '/layer2/layer2.0/Add', '/layer2/layer2.0/relu_1/Relu', '/layer2/layer2.1/conv1/Conv', '/layer2/layer2.1/relu/Relu', '/layer2/layer2.1/conv2/Conv', '/layer2/layer2.1/Add', '/layer2/layer2.1/relu_1/Relu', '/layer3/layer3.0/conv1/Conv', '/layer3/layer3.0/relu/Relu', '/layer3/layer3.0/conv2/Conv', '/layer3/layer3.0/downsample/downsample.0/Conv', '/layer3/layer3.0/Add', '/layer3/layer3.0/relu_1/Relu', '/layer3/layer3.1/conv1/Conv', '/layer3/layer3.1/relu/Relu', '/layer3/layer3.1/conv2/Conv'

### Extraccion de pesos

#### Función *get_inputs*

Esta función toma dos argumentos: la ruta de un modelo en formato ONNX (onnx_model_path) y el nombre de una capa específica (layer_name). Su objetivo es obtener los nombres de las entradas (inputs) de una capa particular en el modelo.

1. Cargar el modelo ONNX: Utiliza la función onnx.load para cargar el modelo desde el archivo especificado en onnx_model_path.

2. Obtener el grafo del modelo: Accede al grafo del modelo ONNX a través del atributo graph del objeto model.

3. Inicializar una lista: Crea una lista vacía llamada weights_values que se utilizará para almacenar los nombres de los inputs de la capa especificada.

4. Iterar sobre las operaciones del grafo: Recorre cada nodo (node) en el grafo del modelo. Cada nodo representa una operación o capa en el modelo.

    * Verificar el nombre de la capa: Compara el nombre del nodo actual (node.name) con el nombre de la capa especificada (layer_name). 

    * Si el nombre coincide, significa que hemos encontrado la capa de interés.

    * Obtener los inputs de la capa: Extrae los nombres de los inputs (node.input) de la capa encontrada y los asigna a la variable inputs.

    * Retornar los inputs: Retorna la lista inputs, que contiene los nombres de los inputs de la capa especificada.

Esta función es útil cuando necesitas conocer qué nodos o tensores sirven como entradas para una capa específica en un modelo ONNX. Esto puede ser relevante para entender las dependencias de datos entre las capas, realizar operaciones de preprocesamiento o incluso para depuración y análisis del modelo.

In [20]:
def get_inputs(onnx_model_path, layer_name):
    # Cargar el modelo ONNX
    model = onnx.load(onnx_model_path)

    # Obtener el grafo del modelo
    graph = model.graph

    # Iterar sobre las operaciones del grafo
    for node in graph.node:
        # Verificar si el nombre de la operación corresponde a la capa
        if node.name == layer_name:
            # print(node.input)
            inputs = node.input
            return inputs
        else:
            return "Not found layer"
        

layer_names = get_layer_names("../models/resnet18.onnx")
get_inputs("../models/resnet18.onnx", layer_names[0])

['input.1', 'onnx::Conv_193', 'onnx::Conv_194']

#### Función *input_to_value*

Esta función toma dos argumentos: la ruta de un modelo en formato ONNX (onnx_model_path) y el nombre de un input específico (input_name). Su propósito es buscar el valor correspondiente al nombre del input dado dentro de los inicializadores del modelo ONNX.

1. Cargar el modelo ONNX: Utiliza la función onnx.load para cargar el modelo desde el archivo especificado en onnx_model_path.

2. Obtener el grafo del modelo: Accede al grafo del modelo ONNX a través del atributo graph del objeto model.

3. Inicializar una variable: Crea una variable llamada input_to_value y la inicializa como None. Esta variable se utilizará para almacenar el valor del input correspondiente.

4. Iterar sobre los inicializadores del grafo: Recorre cada tensor de inicialización (init_tensor) en el grafo del modelo. Los tensores de inicialización son tensores que contienen valores constantes utilizados por el modelo.

    * Verificar el nombre del tensor: Compara el nombre del tensor actual (init_tensor.name) con el nombre del input especificado (input_name).

    * Si el nombre del tensor contiene el nombre del input (input_name in init_tensor.name), significa que hemos encontrado el tensor de interés.

    * Convertir el tensor a un array de numpy: Utiliza la función onnx.numpy_helper.to_array para convertir el tensor de inicialización a un array de numpy.

    * Almacenar el valor: Asigna el array de numpy resultante a la variable input_to_value.

5. Retornar el valor del input: Finalmente, la función retorna el valor almacenado en input_to_value, que corresponde al valor del input especificado.

Esta función es útil cuando necesitas obtener el valor numérico de un input específico en un modelo ONNX. Puede ser útil para inspeccionar los valores iniciales de los inputs, entender mejor el modelo o realizar análisis y depuración.

In [21]:
# Busca el valor correspondiente al nombre del input dado
def input_to_value(onnx_model_path, input_name):
    # Cargar el modelo ONNX
    model = onnx.load(onnx_model_path)

    # Obtener el grafo del modelo
    graph = model.graph

    # Inicializar una variable para almacenar los valores de los pesos
    value = None

    # Iterar sobre los inicializadores del grafo
    for init_tensor in graph.initializer:
        if input_name in init_tensor.name:
            # Convertir el tensor de inicialización a un array de numpy
            value = onnx.numpy_helper.to_array(init_tensor)
            return value
    print("Not found value")



input_to_value("../models/resnet18.onnx", "onnx::Conv_194")

array([ 2.3007244e-01,  2.5738138e-01, -1.0542947e-06, -6.4016706e-01,
       -1.6570578e-08,  1.6138573e-01,  4.5903701e-01, -4.3019020e-07,
        3.0177879e-01, -8.0053715e-06,  3.4324300e-01,  3.0664262e-01,
       -2.4109884e-01, -3.4744200e-05,  1.0718032e-01,  2.1750593e-01,
        3.7091398e-01, -5.4999381e-01, -6.3893002e-01,  5.7853258e-01,
        3.0524322e-01,  5.7428867e-01,  4.8838082e-01,  3.3218446e-01,
        1.9620654e-01,  1.9160990e-01,  1.5354343e-01,  9.8072827e-02,
        4.9628440e-01,  2.5027011e-02,  1.6462386e-01,  3.2675287e-01,
        2.5834763e-01,  4.4402748e-01, -3.0292234e-01, -2.0095145e-02,
       -2.4506735e-07,  3.1680599e-01, -4.9151708e-08,  2.3523429e-01,
        2.3451787e-01,  3.1193152e-01,  4.3329248e-01,  2.8842753e-01,
        2.5865865e-01,  6.9328916e-01,  4.2191592e-01,  3.4001315e-01,
       -8.6909168e-08,  2.5096044e-01,  3.0446300e-01,  6.2337637e-01,
        3.8931307e-01,  3.2796460e-01, -4.0447852e-01,  3.8203886e-01,
      

In [ ]:
def get_type(onnx_model_path, layer_name):
    # Cargar el modelo
    model = onnx.load(onnx_model_path)

    # Obtener el grafo del modelo
    graph = model.graph

    # Iterar sobre las operaciones del grafo
    for node in graph.node:
        # Verificar si el nombre de la operación corresponde a la capa
            if node.name == layer_name:
                  return node.op_type


get_type("../models/resnet18.onnx", layer_names[2])

# Deprecate functions

#### Función *get_weights*

Esta función toma tres argumentos: la ruta de un modelo en formato ONNX (onnx_model_path), el nombre de una capa específica (layer) y un tipo de parámetro (kind) que indica qué tipo de peso o valor se desea obtener.

1. Obtener los inputs de la capa: Utiliza la función get_inputs para obtener los nombres de los inputs de la capa especificada (layer) en el modelo ONNX.

2. Imprimir los inputs: Imprime los nombres de los inputs obtenidos para depuración y seguimiento.

In [9]:
def get_weights(onnx_model_path, layer, kind):
    inputs = get_inputs(onnx_model_path, layer)
    # print(inputs)

    if kind == "x" or kind == "X":
        x = input_to_value(onnx_model_path, inputs[0])
        return x

    elif (kind == "w" or kind == "W") and len(inputs)>1:
        w = input_to_value(onnx_model_path, inputs[1])
        return w

    elif (kind == "b" or kind == "B") and len(inputs)>2:
        b = input_to_value(onnx_model_path, inputs[2])
        return b

    elif kind == "a" or kind == "A":
        all_vals = []
        # Iterar sobre las entradas de la capa
        for input in inputs:
            all_vals.append(input_to_value(onnx_model_path, input))

        return all_vals
    else:
      print(f"The param {kind} is invalid")
    
    # Agregar un mensaje de error si kind no es válido
    raise ValueError(f"El parámetro 'kind' debe ser 'x', 'w', 'b' o 'a', pero se recibió: {kind}")

3. Selección basada en el tipo de parámetro (kind):

    * Inputs: Si kind es "x", se obtiene el valor del primer input (inputs[0]) utilizando la función input_to_value.

    * Weights: Si kind es "w" y hay más de un input, se obtiene el valor del segundo input (inputs[1]).

    * Bias: Si kind es "b" y hay más de dos inputs, se obtiene el valor del tercer input (inputs[2]).

    * All: Si kind es "a", se obtienen todos los valores de los inputs y se almacenan en una lista.

4. Retorno de valores: La función retorna el valor obtenido según el tipo de kind.

In [10]:
# Usa el parametro b para acceder a los bias de la capa
b = get_weights("../models/resnet18.onnx", layer_names[0], "b")
print(b)

[ 2.3007244e-01  2.5738138e-01 -1.0542947e-06 -6.4016706e-01
 -1.6570578e-08  1.6138573e-01  4.5903701e-01 -4.3019020e-07
  3.0177879e-01 -8.0053715e-06  3.4324300e-01  3.0664262e-01
 -2.4109884e-01 -3.4744200e-05  1.0718032e-01  2.1750593e-01
  3.7091398e-01 -5.4999381e-01 -6.3893002e-01  5.7853258e-01
  3.0524322e-01  5.7428867e-01  4.8838082e-01  3.3218446e-01
  1.9620654e-01  1.9160990e-01  1.5354343e-01  9.8072827e-02
  4.9628440e-01  2.5027011e-02  1.6462386e-01  3.2675287e-01
  2.5834763e-01  4.4402748e-01 -3.0292234e-01 -2.0095145e-02
 -2.4506735e-07  3.1680599e-01 -4.9151708e-08  2.3523429e-01
  2.3451787e-01  3.1193152e-01  4.3329248e-01  2.8842753e-01
  2.5865865e-01  6.9328916e-01  4.2191592e-01  3.4001315e-01
 -8.6909168e-08  2.5096044e-01  3.0446300e-01  6.2337637e-01
  3.8931307e-01  3.2796460e-01 -4.0447852e-01  3.8203886e-01
  1.8151292e-01  2.5200108e-01 -4.2829153e-01  2.1370429e-01
  5.7651645e-01  5.6306273e-01 -3.9148784e-01  2.3681219e-01]


In [11]:

# Usa el parametro w para acceder a los pesos de la capa
w = get_weights("../models/resnet18.onnx", layer_names[0], "w")
print(w)

[[[[-2.42673513e-03 -1.42902345e-03 -4.21508710e-04 ...  1.31859714e-02
     3.97881959e-03 -2.95648933e-03]
   [ 2.58123525e-03  2.21903459e-03 -2.56027039e-02 ... -6.31730184e-02
    -3.00623961e-02  8.71622586e-04]
   [-1.61715201e-03  1.37622114e-02  6.88197538e-02 ...  1.21045955e-01
     5.96996024e-02  1.48064643e-02]
   ...
   [-6.41302997e-03  3.73700587e-03  1.69078484e-02 ... -7.75228292e-02
    -9.79548320e-02 -6.00467734e-02]
   [ 7.12986942e-03  9.53979511e-03  1.46381026e-02 ...  9.63860527e-02
     9.16695297e-02  3.86761054e-02]
   [-3.19923507e-03 -8.55828403e-04 -5.60940569e-03 ... -3.50988992e-02
    -1.91518497e-02 -1.34684669e-03]]

  [[-2.65449309e-03 -6.19975710e-03 -8.06808099e-03 ...  7.57427514e-03
     1.54232810e-04 -5.99582680e-03]
   [ 1.06407115e-02  7.82646332e-03 -2.43451148e-02 ... -7.27898702e-02
    -3.73830721e-02 -2.98718543e-04]
   [-1.95012020e-04  2.29226574e-02  9.36528444e-02 ...  1.64871752e-01
     8.59125257e-02  2.90093981e-02]
   ...
   

In [12]:

# Usa el parametro b para acceder a los valores de bias
x = get_weights("../models/resnet18.onnx", layer_names[3], "x")
print(x)

None
